<a href="https://colab.research.google.com/github/sophiabj/03-presidential-election/blob/master/Election_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!python -m pip install --user --upgrade pip

!pip3 install pandas==0.23.4 matplotlib==3.0.3 scipy==1.2.1 scikit-learn==0.22  --user

     |████████████████████████████████| 1.5MB 2.8MB/s 
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 8.9 MB 2.8 MB/s 
     |████████████████████████████████| 13.0 MB 43.3 MB/s 
     |████████████████████████████████| 24.8 MB 1.2 MB/s 
     |████████████████████████████████| 7.0 MB 62.9 MB/s 
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

xarray 0.15.1 requires pandas>=0.25, but you'll have pandas 0.23.4 which is incompatible.
umap-learn 0.4.6 requires s

In [1]:
!pip3 install kfp --upgrade --user

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
     |████████████████████████████████| 116 kB 2.8 MB/s 
     |████████████████████████████████| 1.5 MB 8.8 MB/s 
     |████████████████████████████████| 54 kB 1.4 MB/s 
     |████████████████████████████████| 52 kB 628 kB/s 
     |████████████████████████████████| 56 kB 3.1 MB/s 
     |████████████████████████████████| 200 kB 19.9 MB/s 
  Created wheel for kfp: filename=kfp-1.0.3-py3-none-any.whl size=159872 sha256=7051e2af2e17d946ba51c5d568754a726156ecf82f8cf255c5197ff3900513b0
  Stored in directory: /root/.cache/pip/wheels/6f/17/2a/f0aa226d91bef376b3c97933112e9f341415ccd2488863343b
  Created wheel for kfp-server-api: filename=kfp_server_api-1.0.3-py3-none-any.whl size=104114 sha256=5f3426eed5a2f4319c95370aa2710f2ed064af75644dad3af327df95c72c1b1f
  Stored in directory: /root/.cache/pip/wheels/a3/a7/2f/

In [2]:
!which dsl-compile

In [3]:
import kfp
import kfp.dsl as dsl
import kfp.components as comp

In [4]:
out_dir = "/home/"

In [5]:
def train(data_path, model_file):
    
    import pickle
    import sys, subprocess;
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'pandas==0.23.4'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==0.22'])
    import pandas as pd
    import numpy as np
    
    import os
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.multioutput import MultiOutputRegressor
    from sklearn.preprocessing import MinMaxScaler
    from sklearn import metrics
    
    df = pd.read_csv('https://github.com/sophiabj/03-presidential-election/blob/master/data/usa-2016-presidential-election-by-county.csv')

    new_dataframe = df[['Asian','At Least High School Diploma','Black','Child.Poverty.living.in.families.below.the.poverty.line',
                      'Democrats 08 (Votes)','Democrats 12 (Votes)','Democrats 2008','Democrats 2012','Graduate Degree',
                      'Nearest County','Poverty.Rate.below.federal.poverty.threshold','Republicans 08 (Votes)',
                      'Republicans 12 (Votes)','Republicans 2008','Republicans 2012','Total Population','Votes',
                      'White','White  Asian','total08','total12','total16', 'Democrats 2016', 'Republicans 2016']]
    
    
    
    X = new_dataframe.drop(columns=['Democrats 2016', 'Republicans 2016'])
    y = new_dataframe[['Democrats 2016', 'Republicans 2016']]

    x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    
    scaler = MinMaxScaler()

    normalised_train_df = scaler.fit_transform(x_train)
    normalised_train_df = pd.DataFrame(normalised_train_df, columns=x_train.columns)

    normalised_test_df = scaler.transform(x_test)
    normalised_test_df = pd.DataFrame(normalised_test_df, columns=x_train.columns)


    max_depth = 30
    regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=100,
    max_depth=max_depth,
    random_state=0))
    regr_multirf.fit(normalised_train_df, y_train)

    regr_rf = RandomForestRegressor(n_estimators=100, max_depth=max_depth,
    random_state=2)
    regr_rf.fit(normalised_train_df, y_train)
    
    
     #output file to path
    np.savez_compressed(f'{data_path}/preprocessed-data.npz', 
                       xtrain=normalised_train_df,
                       xtest=normalised_test_df,
                       ytrain=y_train,
                       ytest=y_test)
    print("Preprocessing Done")

     #Save the model to the designated 
    with open(f'{data_path}/{log_reg_file}', 'wb') as file:
        pickle.dump(Log_reg, file)
        
    print("Model Trained")

In [8]:
def predict(data_path, model_file):
    
    import pickle
    import numpy as np
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import recall_score, accuracy_score
    
    with open(f'{data_path}/{log_reg_file}','rb') as file:
        log_reg = pickle.load(file)

    # Load and unpack the test_data
    preprocessed_data = np.load(f'{data_path}/preprocessed-data.npz')
    x_test = preprocessed_data['xtest']
    y_test = preprocessed_data['ytest']

    #Evaluate the model and print results
    y_multirf = regr_multirf.predict(normalised_test_df)
    y_rf = regr_rf.predict(normalised_test_df)

    r2 = metrics.r2_score(y_test,y_multirf)
    round(r2,2)
    
    print('Model \nr2 score = {}' .format(metrics.r2_score(y_test,y_multirf))
    
    #np.savetxt(f'{data_path}/model_result.txt', clf_pred, fmt='%1.2f')
    with open(f'{data_path}/model_result.txt', 'w') as result:
        result.write(" Prediction: {},\nActual: {} ".format(log_reg_pred,y_test))
    
    print('Prediction has been saved successfully!')
    

In [9]:
train_op = comp.func_to_container_op(train , base_image = "python:3.7-slim")
predict_op = comp.func_to_container_op(predict , base_image = "python:3.7-slim")

In [10]:
client = kfp.Client()

Failed to load kube config.


In [11]:
# Define the pipeline
@dsl.pipeline(
   name='Presidential Elections Pipeline',
   description='An ML pipeline that predicts outcome of presidential elections.'
)

# Define parameters to be fed into pipeline
def presidential_election_pipeline(
    data_path: str,
    log_reg_file: str
):
    
    # Define volume to share data between components.
    vop = dsl.VolumeOp(
    name="create_volume",
    resource_name="data-volume", 
    size="1Gi", 
    modes=dsl.VOLUME_MODE_RWO)
    
    # Create presidential elections training component.
    presidential_elections_training_container = train_op(data_path, model_file) \
                                    .add_pvolumes({data_path: vop.volume})

    # Create presidential elections prediction component.
    presidential_elections_predict_container = predict_op(data_path, model_file) \
                                    .add_pvolumes({data_path: presidential_elections_training_container.pvolume})
    
    # Print the result of the prediction
    presidential_elections_result_container = dsl.ContainerOp(
        name="print_prediction",
        image='library/bash:4.4.23',
        pvolumes={data_path: presidential_elections_predict_container.pvolume},
        arguments=['cat', f'{data_path}/result.txt']
    )

In [12]:
DATA_PATH = '/mnt'
MODEL_PATH='presidential_elections_model.h5'

In [13]:
pipeline_func = presidential_elections_container_pipeline

NameError: ignored

In [ ]:
run_name = pipeline_func.__name__ + ' run'

arguments = {"data_path":DATA_PATH,
             "model_file":MODEL_PATH}

# Compile pipeline to generate compressed YAML definition of the pipeline.
kfp.compiler.Compiler().compile(pipeline_func,  
  '{}.zip'.format(experiment_name))

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)